Assume the data has been loaded from 
http://files.grouplens.org/datasets/movielens/ml-20m.zip and unpacked in data/ml-20m

In [59]:
import scipy.io as sio
import pandas as pd
from scipy.sparse import dok_matrix
from random import sample
random.seed(123)

In [11]:
#read movies and tags
movies = pd.read_csv("data/ml-20m/movies.csv", quotechar='"')
tags = pd.read_csv("data/ml-20m/tags.csv", quotechar='"')

In [13]:
movies_dict = dict((m, i) for (i, m) in enumerate(movies['movieId']))
genres = movies['genres'].map(lambda x: set(x.split('|')))
unique_genres = reduce(lambda a,b: a|b, genres, set())
unique_genres = filter(lambda x: x!='(no genres listed)', unique_genres)
genres_dict = dict((t, i) for (i, t) in enumerate(unique_genres))

tags_id_dict = dict((t, i) for (i, t) in enumerate(tags['tag'].unique()))

In [15]:
nm = len(movies)
nug = len(unique_genres)
nut = len(tags_id_dict)
nt = len(tags)
mat = dok_matrix((nm, nug + nt))

for i, gs in enumerate(genres):
    for g in gs:
        if g in genres_dict:
            mat[i,genres_dict[g]] = 1
            
for i in xrange(nt):
    mid = movies_dict[tags['movieId'][i]]
    t = tags['tag'][i]
    ti = nug + tags_id_dict[t]
    mat[mid, ti] += 1    

In [16]:
sio.mmwrite('movies_tags.mtx', mat)

In [3]:
# sample ratings
ratings = pd.read_csv("data/ml-20m/ratings.csv", quotechar='"')
nr = len(ratings)

In [30]:
users = ratings['userId'].unique()
nu = len(users)
user_dict = dict((u, i) for (i, u) in enumerate(users))

In [24]:
nr = len(ratings)
sample(range(nr), nr/5)

[19513288,
 17714416,
 12513412,
 333564,
 11902459,
 18779872,
 18579131,
 2090933,
 9094383,
 4095946,
 2894512,
 18882247,
 13632574,
 331975,
 13180994,
 17744356,
 7224186,
 2955269,
 644257,
 4294091,
 14490204,
 5492976,
 5876911,
 16852158,
 395772,
 12522361,
 16437542,
 11441419,
 621126,
 16882086,
 5587526,
 555006,
 15803218,
 10474346,
 17736224,
 4801797,
 5397152,
 1482306,
 3066303,
 7583174,
 1268042,
 2492484,
 769821,
 4549148,
 2593033,
 7973458,
 11838471,
 6168678,
 5496352,
 18193886,
 18999141,
 16686279,
 5935981,
 16667138,
 8081528,
 15808255,
 18588507,
 9452905,
 7449692,
 10920635,
 8938728,
 3877284,
 2988260,
 3628827,
 5848452,
 6567377,
 6436285,
 13282730,
 13182445,
 2036476,
 18546094,
 14669965,
 17741494,
 8957776,
 12760475,
 19196033,
 10618201,
 14096431,
 2092503,
 13599544,
 6439133,
 16425888,
 18649571,
 15384523,
 4042870,
 17453376,
 15178336,
 16595297,
 4355115,
 19068353,
 9007782,
 9047585,
 19509796,
 2132130,
 19026426,
 9543501,
 

In [54]:
def write_ratings(ratings, samples, name):
    mat = dok_matrix((nu, nm))
    for t, i in enumerate(samples):
        if t % 100000 == 0: print t
        uid = user_dict[ratings['userId'][i]]
        mid = movies_dict[ratings['movieId'][i]]
        rating = ratings['rating'][i]
        mat[uid,mid] = rating
    sio.mmwrite(name, mat)
    print "done writing", name
    return mat

In [55]:
# small debug set
n_debug = nr/5
debug_sample = sample(range(nr), n_debug)
debug_train_sample = debug_sample[0: -n_debug/5]
debug_test_sample = debug_sample[-n_debug/5:-1]

In [56]:
write_ratings(ratings, debug_train_sample, "ratings_debug_train.mtx")
write_ratings(ratings, debug_test_sample, "ratings_debug_test.mtx")

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
done writing ratings_debug_test.mtx
0
100000
200000
300000
400000
500000
600000
700000
800000
done writing ratings_debug_train.mtx


<138493x27278 sparse matrix of type '<type 'numpy.float64'>'
	with 800010 stored elements in Dictionary Of Keys format>

In [57]:
# full train / test set
n_test = nr/5
test_sample = sample(range(nr), n_test)
train_sample = list(set(range(nr)) - set(test_sample))

In [58]:
write_ratings(ratings, train_sample, "ratings_train.mtx")
write_ratings(ratings, test_sample, "ratings_test.mtx")

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

<138493x27278 sparse matrix of type '<type 'numpy.float64'>'
	with 4000052 stored elements in Dictionary Of Keys format>